# P-Multigrid + Jacobi Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

## Smoothing Factor

In [ ]:
# setup
finep = 5
coarsep = 3
dimension = 2
numbercomponents = 1
mesh = Mesh2D(1.0, 1.0)

ctofbasis = TensorH1LagrangePProlongationBasis(coarsep, finep, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep, finep, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep, finep, mesh)

# Jacobi smoother
jacobi = Jacobi(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, jacobi, [ctofbasis])

In [ ]:
# full operator symbols
numberruns = 200
numberrunsω = 50
θ_min = -π/2
θ_max = 3π/2
ω_min = 0.0
ω_max = 1.1

# compute and plot smoothing factor
# setup
eigenvalues = zeros(numberrunsω)

# compute
for w in 1:numberrunsω
    ω = [ω_min + (ω_max - ω_min)*w/numberrunsω]
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns,
        ]
        if abs(θ[1] % 2π) >  π/256
            A = computesymbols(multigrid, ω, [1, 1], θ)
            currenteigenvalues = [abs(val) for val in eigvals(A)]
            eigenvalues[w] = max(eigenvalues[w], currenteigenvalues...)
        end
    end
end

# plot
xrange = ω_min:(ω_max - ω_min)/(numberrunsω-1):ω_max
plot(
    xrange,
    xlabel="ω",
    xtickfont=font(12, "Courier"),
    eigenvalues,
    ytickfont=font(12, "Courier"),
    ylabel="spectral radius",
    linewidth=3,
    legend=:none,
    title="Two-Grid Convergence Factor",
    palette=palette(:tab10)
)
ylims!(min(0.0, eigenvalues...) * 1.1, max(eigenvalues...) * 1.1)

In [ ]:
savefig("two_grid_converge_5_to_3_2d")